In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy.io.wavfile as wavfile
import scipy.signal

from util.plotting import compute_fft_plot_from_sample_rate
from util.data_io import read_rtl_raw_data, read_gqrx_raw_data
from util.filtering import low_pass_filter_complex_signal, band_pass_filter_complex_signal, low_pass_filter_real_signal
from util.phase_lock_loop import phase_lock_loop

# Read an FM broadcast and use standard demodulation

We'll read the file, crop to 5s (for speed), LPF, detect FM using angle differencing.
Then, we detect and sync to the pilot tone. Finally, we detect the stereo difference channel and save

In [ ]:
fs = 2048000
center_frequency = 98500000

fm_signal = read_rtl_raw_data("/home/dominic/radio/data/long_test.raw")
fm_signal = fm_signal[:fs*5]
len(fm_signal) / fs

In [ ]:
filtered_signal = low_pass_filter_complex_signal(signal=fm_signal, cutoff_frequency=55E3, sample_rate=fs)
filtered_fs = fs / 10
filtered_signal = filtered_signal[::10]

angle_diff = np.angle(
    np.conjugate(filtered_signal[:-1]) * filtered_signal[1:]
)

## View the FM spectrum

In [ ]:
f,m = compute_fft_plot_from_sample_rate(angle_diff, filtered_fs)

plt.plot(f,m)

## Extract the pilot tone carrier wave

In [ ]:
# TODO(dominic): Move this into a utility.
filter_coefficients = scipy.signal.firwin(numtaps=129, cutoff=(18E3, 20E3), window="hamming", fs=filtered_fs, pass_zero="bandpass")

# NOTE(dominic): It's important to use a linear phase filter, or do acausal filtering like we do
# below. This is so that after filtering, we get a sinusoid that is exactly in-phase with the
# original.
pilot_signal = scipy.signal.filtfilt(b=filter_coefficients, a=1, x=angle_diff)

## Use a PLL to synchronize to the pilot tone

In [ ]:
# We are searching for a 19khz signal and we want to generate a 38khz reference.
pilot_pll_recovered, pilot_pll_recovered_harmonic, _ = phase_lock_loop(
    pilot_signal, fs=filtered_fs, initial_frequency_estimate=19E3, frequency_bandwidth=250,
    output_frequency_multiplier=2
)

## Check the pilot tone phase with the PLL output

In [ ]:
start_idx = int(filtered_fs*1.0)
end_idx = int(filtered_fs*1.001)

scale = np.mean(np.abs(np.real(pilot_signal[start_idx:end_idx])))

plt.figure(figsize=(10,10))
plt.plot(pilot_signal[start_idx:end_idx])
plt.plot(np.real(pilot_pll_recovered[start_idx:end_idx]*scale))
plt.legend()

## Extract the stereo difference signal and save the audio

In [ ]:
stereo_difference = np.real(angle_diff * np.conjugate(pilot_pll_recovered_harmonic))
stereo_sum = angle_diff

stereo_left = stereo_sum + stereo_difference  # L+R + L-R = 2L
stereo_right = stereo_sum - stereo_difference  # L+R - (L-R) = 2R


# Low-pass filter both stereo channels.
stereo_left = low_pass_filter_real_signal(stereo_left, cutoff_frequency=15E3, sample_rate=filtered_fs)
stereo_right = low_pass_filter_real_signal(stereo_right, cutoff_frequency=15E3, sample_rate=filtered_fs)

audio_downsample_factor = 4
stereo_left = stereo_left[::audio_downsample_factor]
stereo_right = stereo_right[::audio_downsample_factor]
audio_fs = filtered_fs / audio_downsample_factor

In [ ]:
f,m = compute_fft_plot_from_sample_rate(stereo_left, audio_fs)

# plt.plot(f,m)
fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
f,m = compute_fft_plot_from_sample_rate(stereo_right, audio_fs)


fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()
# plt.plot(f,m)

In [ ]:
wavfile.write("data/stereo_sum.wav", int(filtered_fs), stereo_sum)
wavfile.write("data/stereo_difference.wav", int(filtered_fs), stereo_difference)


wavfile.write("data/stereo_left.wav", int(audio_fs), stereo_left)
wavfile.write("data/stereo_right.wav", int(audio_fs), stereo_right)

joint_stereo = np.concatenate([np.expand_dims(stereo_left, axis=1), np.expand_dims(stereo_right, axis=1)], axis=1)
wavfile.write("data/stereo_joint.wav", int(audio_fs), joint_stereo)
